In [76]:
import pandas as pd
import glob
pd.set_option("display.max_rows", 30)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_colwidth", 1000)
pd.set_option("display.width", 2000)

In [77]:
import numpy as np
np.set_printoptions(precision=4,suppress=True)
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [78]:
import nltk 

In [79]:
import spacy

In [80]:
from tqdm import tqdm

In [81]:
nlp = spacy.load('de', disable=['parser', 'tagger'])
# FOR FastText vectors
# nlp.vocab.from_disk('./vocab/')

In [82]:
frames = []
for f in glob.glob('../data/*.csv'):
    frames.append(pd.read_csv(f))

In [83]:
raw_df = pd.concat(frames)

In [84]:
raw_df

,SichterGruppe,SichterName,Host,kind,Datetime,Content,Url,Authorid,Tag,entryid,Id,TwistId,TopicId,TopicName,IsInitialEntry,IsPrivate
0,NaN,NaN,twitter.com,1,21.10.2016 23:18:00,HighTechno ! Ich will Claras WG mal so richtig zu #Elektro dancen sehen . https://t.co/oLKYDGHBGQ,https://twitter.com/Carina42008/status/789576493810737152#aid=1394595750,3796107.0,NaN,bc6c4c70-a00d-429d-8bde-4e7afa072a15,31796013,04b9f324-5cee-4dd3-8387-0ff934ec1472,551631,Alle,False,False
1,NaN,NaN,twitter.com,3,21.10.2016 23:18:00,"Yeah , der WG gefällt deine Musik . Hol dir auch superschnelles Internet ! https://t.co/0dPArCUv0b https://t.co/HHRZaAQnte http://pbs.twimg.com/amplify_video_thumb/0123456789001234567890/img/QEQntlwB_ifYH3Gv.jpg",https://twitter.com/telekomerleben/status/789576496071446528#aid=20459730,1060906.0,NaN,5a676fbd-f7c7-45b1-8744-3a7fd8ebc9e9,31795989,04b9f324-5cee-4dd3-8387-0ff934ec1472,551631,Alle,True,False
2,Sichter FB/TW,Hotspot,www.facebook.com,1,24.09.2016 14:57:02,"Muss ich mit Hotspot registrieren wlan to go auf meinem Handy zu gehen , weiß ich nicht , wie es zu tun ich bin Telekom Kunde",https://www.facebook.com/122768271103611/posts/1141448189235609#pid=122768271103611&uid=122768271103611_1141448189235609&aid=992943974115174&vid=2&wid=122768271103611,7707299.0,NaN,94e8d4f1-60c1-45e5-894a-c11a3ff7b655,30744441,0be928df-7378-421e-be6e-07269f1dac19,551631,Alle,True,False
3,NaN,NaN,www.facebook.com,3,24.09.2016 15:12:35,"Hallo Fiona . Wenn Sie für WLAN TO GO angemeldet sind , dann können Sie unsere HotSpots nutzen in dem Sie sich mit Ihrer T-Online E-Mail-Adresse und dem dazugehörigen Passwort ( das gleiche wie für das Kundencenter ) anmelden . Viele Grüße Christian We .",https://www.facebook.com/122768271103611/posts/1141448189235609#pid=122768271103611_1141448189235609&uid=1141448189235609_1141458792567882&cpid=&aid=122768271103611&vid=2&wid=122768271103611,254017.0,NaN,263da5f3-4f0c-4a3e-ac27-358eb033912f,30744872,0be928df-7378-421e-be6e-07269f1dac19,551631,Alle,False,False
4,NaN,NaN,www.facebook.com,4,24.09.2016 15:16:45,"ich bin nicht gemeldet , es ok ich vergesse es",https://www.facebook.com/122768271103611/posts/1141448189235609#pid=122768271103611_1141448189235609&uid=1141448189235609_1141461172567644&cpid=1141448189235609_1141458792567882&aid=992943974115174&vid=2&wid=122768271103611,7707299.0,NaN,51a8de05-48e4-4422-822f-50aba72fb5f5,30745002,0be928df-7378-421e-be6e-07269f1dac19,551631,Alle,False,False
5,Sichter FB/TW,Hotspot,www.facebook.com,4,24.09.2016 15:18:01,"Jetzt muss ich mal eins fragen . Ich kann doch nicht mit Wlan to go alle Hotspots der Telekom kostenfrei nutzen , oder ? Was ich weiss nur die Hotspots der teilnehmenden Partner / Telekomkunden der Fa . Fon",https://www.facebook.com/122768271103611/posts/1141448189235609#pid=122768271103611_1141448189235609&uid=1141448189235609_1141461895900905&cpid=&aid=1096037533756491&vid=2&wid=122768271103611,127469.0,NaN,a255ed96-818f-42b2-84f1-0c0c7e2581da,30745045,0be928df-7378-421e-be6e-07269f1dac19,551631,Alle,False,False
6,NaN,NaN,www.facebook.com,4,24.09.2016 15:18:01,"Jetzt muss ich mal eins fragen . Ich kann doch nicht mit Wlan to go alle Hotspots der Telekom kostenfrei nutzen , oder ? Was ich weiss nur die Hotspots der teilnehmenden Partner / Telekomkunden der Fa . Fon",https://www.facebook.com/122768271103611/posts/1141448189235609#pid=122768271103611_1141448189235609&uid=1141448189235609_1141461895900905&cpid=&aid=1096037533756491&vid=2&wid=122768271103611,127469.0,NaN,a255ed96-818f-42b2-84f1-0c0c7e2581da,30745045,0be928df-7378-421e-be6e-07269f1dac19,551631,Alle,False,False
7,NaN,NaN,www.facebook.com,4,24.09.2016 15:19:10,ich weis nicht wie es geht :(,https://www.facebook.com/122768271103611/posts/1141448189235609#pid=122768271103611_1141448189235609&uid=1141448189235609_1141462655900829&cpid=1141448189235609_1141458792567882&aid=992943974115174&vid=2&wid=122768271103611,7707299.0,NaN,53632505-a61d-423e-bfc2-9f2047ad75b1,30745129,0be928df-7378-421e-be6e-

In [85]:
pd.unique(raw_df.SichterName)

array([nan, 'Hotspot', 'Festnetz', 'Mobilfunk', 'GK', 'MMS', 'A-Team',
       'Mobile', 'MMS COM', 'Mobilfunk (Mark./T-Shop)', 'Mobilfunk COM',
       'Festnetz (Mark./T-Shop)', 'Festnetz COM', 'MMS (Mark./T-Shop)',
       'Hybrid FB/TW', 'Wechsler', 'Smart Home FB/TW', 'Meinungsbildner',
       'Hosting FB/TW', 'Hybrid & LTE', 'Abuse/Missbrauch',
       'Anfragen in Arabisch', 'Entertain', 'IP & FTTH & PSTN',
       'Neue Themen', 'Dienste & Abos', 'Hosting', 'Endgeräte'],
      dtype=object)

In [86]:
df = pd.concat([raw_df[raw_df.Tag.isnull()], raw_df[raw_df.Tag=='Prio-Fall']])
df = df[df.kind == 1]

In [87]:
import re
GRUBER_URLINTEXT_PAT = re.compile(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?\xab\xbb\u201c\u201d\u2018\u2019]))')
NUMBER_CLEAN = re.compile(r'\d+(:?[.,]?\d+)*')
TIME_CLEAN = re.compile(r'\d+:\d+(?:\s?[Uu]hr)?')
PUNCT_CLEAN = re.compile(r'(\s?)[\.\,()\[\]\{\}\^\:\'\"\/;]+(\s?)')
DASH_CLEAN = re.compile(r'([^eEtT])[-]')
TWEET_CLEAN = re.compile(r'@\w+')
HASH_CLEAN = re.compile(r'#(\w+)')

def preprocessing(text):
    try:
        # clean-up url
        text = GRUBER_URLINTEXT_PAT.sub('url', text)
        # remove time
        text = TIME_CLEAN.sub('zeit', text)
        # clean-up numbers
        text = NUMBER_CLEAN.sub('nummer', text)
        text = PUNCT_CLEAN.sub(r'\1\2',text)
        text = TWEET_CLEAN.sub('', text)
        text = HASH_CLEAN.sub(r'\1', text)
        text = DASH_CLEAN.sub(r'\1 ', text)
        return text
    except:
        return ''

with open('./stopwords-de.txt') as f:
    stopwords = f.read().split('\n')
    
def filter_toks(token):
    if token.tag_ in ['$,','$.','$(']:
        return False
    if token.pos_ in ['ADP']:
        return False
    if token.lemma_.lower() in stopwords:
        return False
    return True

In [88]:
documents = [] # Content
docs = [] # Content through spacy
labels = [] # SichterName
tags = [] # Tag

for (idx, row) in tqdm(df.iterrows()):
    labels.append(row.SichterName)
    documents.append(preprocessing(row.Content))
#     docs.append([token for token in nlp(documents[-1])])
    tags.append(str(row.Tag))

109087it [00:24, 4494.45it/s]


In [90]:
import pickle

In [91]:
# Save processed rows
pickle.dump(documents, open('documents.pl', 'wb'))
pickle.dump(labels, open('labels.pl', 'wb'))
pickle.dump(tags, open('tags.pl', 'wb'))

# RASA TRAINING DATA

In [96]:
import simplejson
fp = {
    "rasa_nlu_data": {
        "common_examples": [],
        "regex_features" : [],
        "entity_synonyms": []
    }
}
fb = {
    "rasa_nlu_data": {
        "common_examples": [],
        "regex_features" : [],
        "entity_synonyms": []
    }
}

for i in range(len(documents)):
    if not documents or documents == '' or len(documents) == 0:
        continue
    fp["rasa_nlu_data"]['common_examples'].append(dict(text=documents[i], intent=str(tags[i]), entities=[]))
    fb["rasa_nlu_data"]['common_examples'].append(dict(text=documents[i], intent=str(labels[i]), entities=[]))

with open('train_prio.json', 'wt') as file_p:
    simplejson.dump(fp, file_p)
    
with open('train_bucket.json', 'w') as file_b:
    simplejson.dump(fb, file_b)